In [19]:
!pip install PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

fileId = drive.CreateFile({'id': '#'}) 
print(fileId['title'])  # dataset.zip
fileId.GetContentFile('temp.zip')  # Save Drive file as a local file

!unzip temp.zip -d ./

imdb_master.zip
Archive:  temp.zip
replace ./imdb_master.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ./imdb_master.csv       


In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
from sklearn.model_selection import train_test_split
nltk.download('stopwords')
nltk.download('wordnet')
from keras.layers.core import Activation
from keras.models import Model
from keras.layers import Input, Embedding, Flatten, dot
from keras import backend as K
from keras.optimizers import Adam

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
df = pd.read_csv('imdb_master.csv', encoding = "ISO-8859-1")
del df['Unnamed: 0']
del df['file']
del df['type']

In [0]:
df = df.loc[0:49999]
df = df.sample(frac=1).reset_index(drop=True)

In [0]:
def clean_reviews(text):
    lemmatizer = WordNetLemmatizer()
    my_stopwords = stopwords.words('english') 
    text = text.replace("<br >", "")
    text = text.replace("</br >", "")        
    text = re.sub('[^a-zA-Z]',' ', text)
    text = text.lower() 
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
    text = [lemmatizer.lemmatize(token, "v") for token in text]
    text = [word for word in text if not word in my_stopwords]
    text = " ".join(text)
    return text  

In [0]:
df["review"]=df.review.apply(lambda x: clean_reviews(x))

In [25]:
df["label"] = df["label"].map({'pos': 1, 'neg': 0})
df.head()

,review,label
0,pure crap probably worst biblical theme film ...,0
1,north africa small arab town edge sahara...,1
2,even theatre person highly recommend see b...,1
3,familiar story man writer sell soul devil or...,0
4,go bother plot synopsis since know movie almos...,0


In [0]:
X_train, X_test, y_train, y_test = train_test_split(df["review"], df["label"], test_size=0.2)


In [0]:
veczr =  CountVectorizer(ngram_range=(1,3), binary=True, token_pattern=r'\w+', max_features=800000)
dtm_train = veczr.fit_transform(X_train)
dtm_test = veczr.transform(X_test)

In [28]:
def dtm2wid(dtm, maxlen=2000):
    x = []
    nwds = []
    for idx, row in enumerate(dtm):
        seq = []
        indices = (row.indices + 1).astype(np.int64)
        np.append(nwds, len(indices))
        data = (row.data).astype(np.int64)
        count_dict = dict(zip(indices, data))
        for k,v in count_dict.items():
            seq.extend([k]*v)
        num_words = len(seq)
        nwds.append(num_words)
        # pad up to maxlen
        if num_words < maxlen: 
            seq = np.pad(seq, (maxlen - num_words, 0), mode='constant')
        # truncate down to maxlen
        else:                  
            seq = seq[-maxlen:]
        x.append(seq)
    nwds = np.array(nwds)
    print('sequence stats: avg:%s, max:%s, min:%s' % (nwds.mean(), nwds.max(), nwds.min()) )
    return np.array(x)

maxlen = 2000
x_train = dtm2wid(dtm_train, maxlen=maxlen)
x_test = dtm2wid(dtm_test, maxlen=maxlen)

sequence stats: avg:192.97095, max:2146, min:5
sequence stats: avg:172.109, max:1224, min:12


In [0]:
def pr(dtm, y, y_i):
    p = dtm[y==y_i].sum(0)
    return (p+1) / ((y==y_i).sum()+1)

y_train = np.array(y_train) 

a = pr(dtm_train, y_train, 1)
nbratios = np.log(pr(dtm_train, y_train, 1)/pr(dtm_train, y_train, 0))
nbratios = np.squeeze(np.asarray(nbratios))

In [0]:
def get_model(num_words, maxlen, nbratios=None):
    embedding_matrix = np.zeros((num_words, 1))
    for i in range(1, num_words): # skip 0, the padding value
        if nbratios is not None:
            # if log-count ratios are supplied, then it's NBSVM
            embedding_matrix[i] = nbratios[i-1]
        else:
            # if log-count rations are not supplied, this reduces to a logistic regression
            embedding_matrix[i] = 1

    # set up the model
    inp = Input(shape=(maxlen,))
    r = Embedding(num_words, 1, input_length=maxlen, weights=[embedding_matrix], trainable=False)(inp)
    x = Embedding(num_words, 1, input_length=maxlen, embeddings_initializer='glorot_normal')(inp)
    x = dot([r,x], axes=1)
    x = Flatten()(x)
    x = Activation('sigmoid')(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy',
                  optimizer=Adam(lr=0.001),
                  metrics=['accuracy'])
    return model

In [35]:
num_words = len([v for k,v in veczr.vocabulary_.items()]) + 1
model = get_model(num_words, maxlen, nbratios=nbratios)
model.fit(x_train, y_train,
          batch_size=32,
          epochs=10,
          validation_data=(x_test, y_test))

Train on 40000 samples, validate on 10000 samples
Epoch 1/10
40000/40000 [==============================] - 25s 617us/step - loss: 0.3098 - acc: 0.9230 - val_loss: 0.2663 - val_acc: 0.9109
Epoch 2/10
40000/40000 [==============================] - 24s 607us/step - loss: 0.1214 - acc: 0.9777 - val_loss: 0.2376 - val_acc: 0.9140
Epoch 3/10
40000/40000 [==============================] - 24s 605us/step - loss: 0.0692 - acc: 0.9918 - val_loss: 0.2262 - val_acc: 0.9148
Epoch 4/10
40000/40000 [==============================] - 24s 601us/step - loss: 0.0431 - acc: 0.9963 - val_loss: 0.2208 - val_acc: 0.9161
Epoch 5/10
40000/40000 [==============================] - 24s 604us/step - loss: 0.0278 - acc: 0.9986 - val_loss: 0.2188 - val_acc: 0.9151
Epoch 6/10
40000/40000 [==============================] - 24s 602us/step - loss: 0.0185 - acc: 0.9993 - val_loss: 0.2189 - val_acc: 0.9154
Epoch 7/10
40000/40000 [==============================] - 24s 603us/step - loss: 0.0125 - acc: 0.9995 - val_loss: 0.

In [0]:
# Thanks to this blog : https://medium.com/@asmaiya/a-neural-implementation-of-nbsvm-in-keras-d4ef8c96cb7c